In [76]:
#Instala os pacotes básicos
import pandas as pd
try:
    import pandas_ta as ta
except:
    !pip install pandas_ta
    import pandas_ta as ta
import datetime as dt
import math
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
import numpy as np

from IPython.display import display

#Constantes e parâmetros
DIRETORIO_DADOS = './data/'         #Diretório dos arquivos de dados (leitura)
DIRETORIO_SAIDA = './output/'       #Diretório dos arquivos de saída (escrita)
TICKET = 'EURUSD'                   #Moeda (par)
TIMEFRAME = 240                     #Tempo gráfico
INICIO = '2015-01-01'               #Data inicial
FINAL = '2024-04-01'                #Data final

ATR_PERIOD = 14                     #Período do ATR   
EMA_FAST = 50                       #Período da EMA rápida
EMA_MEAN = 144                      #Período da EMA média    
EMA_SLOW = 200                      #Período da EMA lenta
RSI_PERIOD = 14                     #Período do RSI 

TAM_STOP = 1                        #Tamanho do stop loss em ATRs
RISCO_RETORNO = 3                   #Relação risco-retorno 
TOTAL_TAXAS = 0.0001                #Soma da taxa, spread e slipagge

In [77]:
#Carrega a base de dados, define o nome das colunas, cria o índcie e remove os registros que não estão no período solicitado
#Reforna o dataframe com os dados carregados no formato exigido pelas rotinas do sistema

# df = pd.read_csv('./Data/GOOG.csv')
# df.dropna(axis=0, how='all', inplace=True) #Remove todos os registros com null em algum campo
# df.columns=(['Date', 'Open', 'High', 'Low', 'Close', 'Volume']) #Seta os cabeçalhos dos campos
# df = df[(df['High'] > df['Low']) & (df['Volume'] > 1)] #Remove candles sem negociação
# df['Open time'] = pd.to_datetime(df['Date'], format='%Y.%m.%d')
# df.set_index('Open time', inplace = True) #Cria o índice

#end def

In [78]:
#Carrega a base de dados, define o nome das colunas, cria o índcie e remove os registros que não estão no período solicitado
#Reforna o dataframe com os dados carregados no formato exigido pelas rotinas do sistema
def carregarArquivo(DIRETORIO, TICKET, TIMEFRAME, INICIO, FINAL):
    df_full = pd.read_csv(DIRETORIO+TICKET+str(TIMEFRAME)+'.csv')
    df_full.dropna(axis=0, how='all', inplace=True) #Remove todos os registros com null em algum campo
    df_full.columns=(['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']) #Seta os cabeçalhos dos campos
    df_full = df_full[(df_full['High'] > df_full['Low']) & (df_full['Volume'] > 1)] #Remove candles sem negociação
    df_full['Open time'] = pd.to_datetime(df_full['Date'] + ' ' + df_full['Time'] + ':00+0300', format='%Y.%m.%d %H:%M:%S%z')
    df_full.drop(df_full.iloc[:, 0:2], axis = 1, inplace=True) #Remove os campos date e time
    df_full.set_index('Open time', inplace = True) #Cria o índice
    df = df_full[(df_full.index >= pd.to_datetime(INICIO+' 00:00:00+0300')) & (df_full.index <= pd.to_datetime(FINAL+' 00:00:00+0300'))]
    del[df_full] #Deleta a base full
    return df #Retorna a base com os registros do período selecionado
# #end def

#Calcula o ATR desde o primeiro ao último registro e insere no dataframe. Recebe o dataframe e adiona fisicamente a coluna 'ATR' nele
#Pressupõe que o dataframe está no formato padrão com as colunas 'Open', 'High', 'Low' e 'Close'
def calcularIndicadorATR(df, period):
    df['ATR'] = ta.atr(open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], length=period)
    df['ATR'].fillna(ta.true_range(open=df['Open'], high=df['High'], low=df['Low'], close=df['Close']).ewm(span=period, 
                adjust=False).mean(), inplace=True) #Preenche os primeiros period registros
    df['ATR'].fillna((df['High']-df['Low']), inplace=True) #Preenche algum residual (em geral o primeiro registro)
    df['ATR'] = round(df['ATR'], 5)
#end def

#Calcula a EMA desde o primeiro ao último registro e insere no dataframe. Recebe o dataframe e adiona fisicamente a coluna 'EMA+period' nele
#Pressupõe que o dataframe está no formato padrão com as colunas 'Open', 'High', 'Low' e 'Close'
def calcularIndicadorEMA(df, period):
    df['EMA'+str(period)] = ta.ema(open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], length=period)
    df['EMA'+str(period)].fillna(df['Close'].ewm(span=period, adjust=False).mean(), inplace=True)
    df['EMA'+str(period)] = round(df['EMA'+str(period)], 5)
#end def

#Calcula o oscilador de Bolllinger e insere ele no dataframe. Recebe o dataframe e adiona fisicamente a coluna 'Boll+period' nele
#Pressupõe que o dataframe está no formato padrão com as colunas 'Open', 'High', 'Low' e 'Close'
def calcularIndicadorBollingerOscilattor(df, period):
    df['Boll'+str(period)] = (df['Close'] - ta.ema(close=df['Close'],length=period)) / ta.stdev(close=df['Close'], length=period)
    df['Boll'+str(period)] = round(df['Boll'+str(period)], 5)
#end def

#Cacula a inclinação (derivada) da EMA e insere ela no dataframe. Recebe o dataframe e adiona fisicamente a coluna 'Deriv+period' nele
#Pressupõe que o dataframe está no formato padrão e possui a coluna 'EMA+period' nele
def calcularIndicadorROC(df, period):
    df['ROC'+str(period)] = ta.roc(df['EMA'+str(period)], max(1, math.trunc(period/4.236+0.5)))
    df['ROC'+str(period)] = round(df['ROC'+str(period)], 5)
#end def

In [79]:
#Carrega o arquivo e calcula os indicadores
df = carregarArquivo(DIRETORIO_DADOS, TICKET, TIMEFRAME, INICIO, FINAL)
calcularIndicadorATR(df, ATR_PERIOD)
calcularIndicadorEMA(df, EMA_FAST)
calcularIndicadorEMA(df, EMA_MEAN) # Nova parametro
calcularIndicadorEMA(df, EMA_SLOW)

calcularIndicadorBollingerOscilattor(df, EMA_FAST)
calcularIndicadorBollingerOscilattor(df, EMA_MEAN)
calcularIndicadorBollingerOscilattor(df, EMA_SLOW)

calcularIndicadorROC(df, EMA_FAST)
calcularIndicadorROC(df, EMA_MEAN)
calcularIndicadorROC(df, EMA_SLOW)

#Cruzamento de médias
df['CrxFM'] = (df['EMA'+str(EMA_FAST)] - df['EMA'+str(EMA_MEAN)]) / df['Close']
df['CrxFS'] = (df['EMA'+str(EMA_FAST)] - df['EMA'+str(EMA_SLOW)]) / df['Close']
df['CrxMS'] = (df['EMA'+str(EMA_MEAN)] - df['EMA'+str(EMA_SLOW)]) / df['Close'] 

#RSI, engolfos bullish (+1), bearish (-1) e FU candles
df['RSI'] = ta.rsi(close=df['Close'], length=RSI_PERIOD)
df['Engolfo'] = np.where(df['Close'] > df['Open'], (np.where(df['Close'] > df['High'].shift(+1), 1, 0)), (np.where(df['Close'] < df['Low'].shift(+1), -1, 0)))
df['FuCandle'] = np.where(df['High']-df['Low'] < 1.618*df['ATR'], 0, np.where((df['Open']-df['Low'] > (df['High']-df['Low'])*0.618) & (df['Close']-df['Low'] > (df['High']-df['Low'])*0.618), +1, 
         np.where((df['High']-df['Close'] > (df['High']-df['Low'])*0.618) & (df['High']-df['Open'] > (df['High']-df['Low'])*0.618), -1, 0))) #Pavio > 61.8% do tamanho do candle (size) e size > 1.618*ATR

#Remove as primeiras velas onde não foi possível calcular os indicadores
# df.dropna(axis=0, how='any', inplace=True)
df

,Open,High,Low,Close,Volume,ATR,EMA50,EMA144,EMA200,Boll50,...,Boll200,ROC50,ROC144,ROC200,CrxFM,CrxFS,CrxMS,RSI,Engolfo,FuCandle
Open time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00+03:00,1.20538,1.20690,1.20344,1.20573,13196,0.00346,1.20573,1.20573,1.20573,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,0,0
2015-01-02 12:00:00+03:00,1.20574,1.20577,1.20250,1.20302,13937,0.00327,1.20562,1.20569,1.20570,NaN,...,NaN,NaN,NaN,NaN,-0.000058,-0.000066,-0.000008,NaN,-1,0
2015-01-02 16:00:00+03:00,1.20301,1.20330,1.20035,1.20092,18268,0.00323,1.20544,1.20563,1.20566,NaN,...,NaN,NaN,NaN,NaN,-0.000158,-0.000183,-0.000025,NaN,-1,0
2015-01-02 20:00:00+03:00,1.20093,1.20150,1.19997,1.20014,7681,0.00300,1.20523,1.20555,1.20560,NaN,...,NaN,NaN,NaN,NaN,-0.000267,-0.000308,-0.000042,NaN,-1,0
2015-01-05 00:00:00+03:00,1.19454,1.19755,1.18642,1.19541,17684,0.00443,1.20485,1.20541,1.20550,NaN,...,NaN,NaN,NaN,NaN,-0.000468,-0.000544,-0.000075,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 08:00:00+03:00,1.00403,1.00460,0.99741,0.99793,23205,0.00411,1.00168,1.01025,1.01431,-0.89784,...,-1.53841,-0.02595,-0.69789,-0.93855,-0.008588,-0.012656,-0.004068,46.462580,-1,0
2022-08-31 12:00:00+03:00,0.99794,1.00184,0.99714,1.00084,28012,0.00416,1.00165,1.01012,1.01417,-0.20233,...,-1.25096,-0.02495,-0.68334,-0.93287,-0.008463,-0.012509,-0.004047,51.560650,0,0
2022-08-31 16:00:00+03:00,1.00085,1.00790,0.99952,1.00535,39166,0.00446,1.00179,1.01005,1.01409,0.92636,...,-0.82045,-0.00898,-0.66092,-0.92133,-0.008216,-0.012235,-0.004019,58.203527,1,0


### Estrutura

In [80]:
#Verifica se o topo mais recente já foi consolidado
def topoConsolidado(df, extremo, pos_atual):
    for j in range(df.index.get_loc(extremo), pos_atual, 1): 
        if df['Close'].iloc[j+1] < df['Low'].iloc[j]: 
            return True #Se engolfou retorna true
    return False
#end def


#Verifica se o fundo mais recente já foi consolidado
def fundoConsolidado(df, extremo, pos_atual):
    for j in range(df.index.get_loc(extremo), pos_atual, 1): 
        if df['Close'].iloc[j+1] > df['High'].iloc[j]:
            return True #Se engolfou retorna true
    return False
#end def

In [81]:
#Variáveis globais usadas daqui em diante
StructTopIdx = StructBottomIdx = df.index[0]

#Cria e inicializa novos campos no dataframe
df['Structure'] = 0
df['StructTop'] = df['High'].loc[StructTopIdx]
df['StructBottom'] = df['Low'].loc[StructBottomIdx]

#Listas e variáveis locais
lista_swings = []
lista_rompimentos = []
lista_demands = []
lista_supplys = []
structure = 0
steadyTop = steadyBottom = False

#Agora faz toda a mágica
for i in range(1, len(df)):
    #Verifica se os topos e fundos foram consolidados
    if not steadyTop: 
        steadyTop = topoConsolidado(df, StructTopIdx, i)
    if not steadyBottom: 
        steadyBottom = fundoConsolidado(df, StructBottomIdx, i)
     
    #Verifica se rompeu o topo
    if df['High'].iloc[i] > df['High'].loc[StructTopIdx]: #Violou o extremo
        if steadyTop: #Se o extremo já está consolidado
            if df['Close'].iloc[i] > df['High'].loc[StructTopIdx]: #Fechou acima do extremo                
                #Salva o rompimento
                lista_rompimentos.append([StructTopIdx, df.index[i], df['High'].loc[StructTopIdx]])
                #Atualiza o fundo (forte)
                StructBottomIdx = df['Low'].iloc[df.index.get_loc(StructTopIdx)+1:i+1].idxmin()
                steadyBottom = True #O fundo é consolidado no rompimento do topo
                lista_swings.append([StructBottomIdx, df['Low'].loc[StructBottomIdx]]) 
                lista_demands.append([StructBottomIdx, df['Low'].loc[StructBottomIdx], df.index[i+1], df['High'].loc[StructBottomIdx]])
                #Guarda o novo topo, marca ele como não consolidado e registra a estrutura como bullish
                StructTopIdx = df.index[i]
                steadyTop = False
                structure = 1 #Bullish
        else: 
            StructTopIdx = df.index[i] #Extremo não consolidado - atualiza mas não registra rompimento
                                  
    #Verifica se rompeu o fundo
    if df['Low'].iloc[i] < df['Low'].loc[StructBottomIdx]: #Violou o extremo     
        if steadyBottom: #Se o extremo já está consolidado
            if df['Close'].iloc[i] < df['Low'].loc[StructBottomIdx]: #Fechou abaixo do extremo
                #Salva o rompimento
                lista_rompimentos.append([StructBottomIdx, df.index[i], df['Low'].loc[StructBottomIdx]])
                #Atualiza o topo (forte)
                StructTopIdx = df['High'].iloc[df.index.get_loc(StructBottomIdx)+1:i+1].idxmax()        
                steadyTop = True #O topo forte é consolidado no rompimento do fundo
                lista_swings.append([StructTopIdx, df['High'].loc[StructTopIdx]]) 
                lista_supplys.append([StructTopIdx, df['Low'].loc[StructTopIdx], df.index[i+1], df['High'].loc[StructTopIdx]])
                #Guarda o novo fundo, marca ele como não consolidado e registra a estrutura como bearish
                StructBottomIdx = df.index[i]
                steadyBottom = False
                structure = -1 #Bearsih
        else: 
            StructBottomIdx = df.index[i] #Extremo não consolidado - atualiza mas não registra rompimento
            
    #Salva a estrutura corrente no dataframe
    df.at[df.index[i], 'Structure'] = structure #Se estamos bullish ou bearish
    df.at[df.index[i], 'StructTop'] = df['High'].loc[StructTopIdx] #Topo atual da estrutra
    df.at[df.index[i], 'StructBottom'] = df['Low'].loc[StructBottomIdx] #Fundo atual da estrutra
#end for

#Premium ou discount? Guardamos em valores entre 0 e 1                                                  
df['StructPD'] = round((df['Close']-df['StructBottom']) / (df['StructTop'] - df['StructBottom']), 6)
    
#Agora criamos as listas de topos e fundos e supplys e demands
df_swings = pd.DataFrame(lista_swings, columns=['Time', 'Value'])
df_rompimentos = pd.DataFrame(lista_rompimentos, columns=['x0', 'x1', 'y'])
df_demands = pd.DataFrame(lista_demands, columns=['x0', 'y0', 'x1', 'y1'])
df_supplys = pd.DataFrame(lista_supplys, columns=['x0', 'y0', 'x1', 'y1'])

#Apaga as listas e variáveis locais
del[lista_swings, lista_rompimentos, lista_demands, lista_supplys]

In [82]:
#Adiciona o campo VelaBull para simplificar os cálculos futuros (true se o candle é de alta)
df['VelaBull'] = df['Close'] > df['Open'] 

#Cálculo dos imbalances
df['ImbLow'] = 0.0
df['ImbHigh'] = 0.0
for i in range(1, len(df)-1):
    if df['VelaBull'].iloc[i]: #Se o candle é bullish
        if df['High'].iloc[i-1] < df['Low'].iloc[i+1]:
            df.at[df.index[i], 'ImbLow'] = df['High'].iloc[i-1]
            df.at[df.index[i], 'ImbHigh'] = df['Low'].iloc[i+1]
    else: #Se o candle é bearish
        if df['Low'].iloc[i-1] > df['High'].iloc[i+1]:
            df.at[df.index[i], 'ImbLow'] = df['High'].iloc[i+1]
            df.at[df.index[i], 'ImbHigh'] = df['Low'].iloc[i-1]
#end for

In [83]:
display(df)

,Open,High,Low,Close,Volume,ATR,EMA50,EMA144,EMA200,Boll50,...,RSI,Engolfo,FuCandle,Structure,StructTop,StructBottom,StructPD,VelaBull,ImbLow,ImbHigh
Open time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00+03:00,1.20538,1.20690,1.20344,1.20573,13196,0.00346,1.20573,1.20573,1.20573,NaN,...,NaN,0,0,0,1.20690,1.20344,0.661850,True,0.00000,0.00000
2015-01-02 12:00:00+03:00,1.20574,1.20577,1.20250,1.20302,13937,0.00327,1.20562,1.20569,1.20570,NaN,...,NaN,-1,0,0,1.20690,1.20250,0.118182,False,1.20330,1.20344
2015-01-02 16:00:00+03:00,1.20301,1.20330,1.20035,1.20092,18268,0.00323,1.20544,1.20563,1.20566,NaN,...,NaN,-1,0,0,1.20690,1.20035,0.087023,False,1.20150,1.20250
2015-01-02 20:00:00+03:00,1.20093,1.20150,1.19997,1.20014,7681,0.00300,1.20523,1.20555,1.20560,NaN,...,NaN,-1,0,0,1.20690,1.19997,0.024531,False,1.19755,1.20035
2015-01-05 00:00:00+03:00,1.19454,1.19755,1.18642,1.19541,17684,0.00443,1.20485,1.20541,1.20550,NaN,...,NaN,0,1,0,1.20690,1.18642,0.438965,True,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 08:00:00+03:00,1.00403,1.00460,0.99741,0.99793,23205,0.00411,1.00168,1.01025,1.01431,-0.89784,...,46.462580,-1,0,-1,1.00955,0.99004,0.404408,False,0.00000,0.00000
2022-08-31 12:00:00+03:00,0.99794,1.00184,0.99714,1.00084,28012,0.00416,1.00165,1.01012,1.01417,-0.20233,...,51.560650,0,0,-1,1.00955,0.99004,0.553562,True,0.00000,0.00000
2022-08-31 16:00:00+03:00,1.00085,1.00790,0.99952,1.00535,39166,0.00446,1.00179,1.01005,1.01409,0.92636,...,58.203527,1,0,-1,1.00955,0.99004,0.784726,True,1.00184,1.00355


In [84]:
#Bollinger Bandas
#calculate bollinger bands

#calcule smd
df['sma'] = df['Close'].rolling(20).mean()

#calculate standart deviation
df['sd'] = df['Close'].rolling(20).std()

#calculate lower band
df['lb'] = df['sma'] - 2 * df['sd']

#calculate upper band
df['ub'] = df['sma'] + 2 * df['sd']

# df.dropna(inplace = True)

In [85]:
#find signals
def find_signal(close, lower_band, upper_band):
    if close < lower_band:
        return -1 #compra
    elif close > upper_band:
        return 1 #venda
    else:
        return 0
    
df['signal_BB'] = np.vectorize(find_signal)(df['Close'], df['lb'], df['ub'])
df

c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2411: RuntimeWarning: invalid value encountered in find_signal (vectorized)
  outputs = ufunc(*inputs)


,Open,High,Low,Close,Volume,ATR,EMA50,EMA144,EMA200,Boll50,...,StructBottom,StructPD,VelaBull,ImbLow,ImbHigh,sma,sd,lb,ub,signal_BB
Open time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00+03:00,1.20538,1.20690,1.20344,1.20573,13196,0.00346,1.20573,1.20573,1.20573,NaN,...,1.20344,0.661850,True,0.00000,0.00000,NaN,NaN,NaN,NaN,0
2015-01-02 12:00:00+03:00,1.20574,1.20577,1.20250,1.20302,13937,0.00327,1.20562,1.20569,1.20570,NaN,...,1.20250,0.118182,False,1.20330,1.20344,NaN,NaN,NaN,NaN,0
2015-01-02 16:00:00+03:00,1.20301,1.20330,1.20035,1.20092,18268,0.00323,1.20544,1.20563,1.20566,NaN,...,1.20035,0.087023,False,1.20150,1.20250,NaN,NaN,NaN,NaN,0
2015-01-02 20:00:00+03:00,1.20093,1.20150,1.19997,1.20014,7681,0.00300,1.20523,1.20555,1.20560,NaN,...,1.19997,0.024531,False,1.19755,1.20035,NaN,NaN,NaN,NaN,0
2015-01-05 00:00:00+03:00,1.19454,1.19755,1.18642,1.19541,17684,0.00443,1.20485,1.20541,1.20550,NaN,...,1.18642,0.438965,True,0.00000,0.00000,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 08:00:00+03:00,1.00403,1.00460,0.99741,0.99793,23205,0.00411,1.00168,1.01025,1.01431,-0.89784,...,0.99004,0.404408,False,0.00000,0.00000,0.999307,0.003522,0.992264,1.006351,0
2022-08-31 12:00:00+03:00,0.99794,1.00184,0.99714,1.00084,28012,0.00416,1.00165,1.01012,1.01417,-0.20233,...,0.99004,0.553562,True,0.00000,0.00000,0.999536,0.003461,0.992614,1.006459,0
2022-08-31 16:00:00+03:00,1.00085,1.00790,0.99952,1.00535,39166,0.00446,1.00179,1.01005,1.01409,0.92636,...,0.99004,0.784726,True,1.00184,1.00355,0.999858,0.003692,0.992475,1.007242,0


In [86]:
import numpy as np

#find signals
def find_signal(rsi_14):
    if rsi_14 > 70:
        return 1
    elif rsi_14 < 30:
        return -1
    else:
        return 0
    
df['signal_rsi'] = np.vectorize(find_signal)(df['RSI'])
df

c:\Users\Marco\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2411: RuntimeWarning: invalid value encountered in find_signal (vectorized)
  outputs = ufunc(*inputs)


,Open,High,Low,Close,Volume,ATR,EMA50,EMA144,EMA200,Boll50,...,StructPD,VelaBull,ImbLow,ImbHigh,sma,sd,lb,ub,signal_BB,signal_rsi
Open time,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00+03:00,1.20538,1.20690,1.20344,1.20573,13196,0.00346,1.20573,1.20573,1.20573,NaN,...,0.661850,True,0.00000,0.00000,NaN,NaN,NaN,NaN,0,0
2015-01-02 12:00:00+03:00,1.20574,1.20577,1.20250,1.20302,13937,0.00327,1.20562,1.20569,1.20570,NaN,...,0.118182,False,1.20330,1.20344,NaN,NaN,NaN,NaN,0,0
2015-01-02 16:00:00+03:00,1.20301,1.20330,1.20035,1.20092,18268,0.00323,1.20544,1.20563,1.20566,NaN,...,0.087023,False,1.20150,1.20250,NaN,NaN,NaN,NaN,0,0
2015-01-02 20:00:00+03:00,1.20093,1.20150,1.19997,1.20014,7681,0.00300,1.20523,1.20555,1.20560,NaN,...,0.024531,False,1.19755,1.20035,NaN,NaN,NaN,NaN,0,0
2015-01-05 00:00:00+03:00,1.19454,1.19755,1.18642,1.19541,17684,0.00443,1.20485,1.20541,1.20550,NaN,...,0.438965,True,0.00000,0.00000,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31 08:00:00+03:00,1.00403,1.00460,0.99741,0.99793,23205,0.00411,1.00168,1.01025,1.01431,-0.89784,...,0.404408,False,0.00000,0.00000,0.999307,0.003522,0.992264,1.006351,0,0
2022-08-31 12:00:00+03:00,0.99794,1.00184,0.99714,1.00084,28012,0.00416,1.00165,1.01012,1.01417,-0.20233,...,0.553562,True,0.00000,0.00000,0.999536,0.003461,0.992614,1.006459,0,0
2022-08-31 16:00:00+03:00,1.00085,1.00790,0.99952,1.00535,39166,0.00446,1.00179,1.01005,1.01409,0.92636,...,0.784726,True,1.00184,1.00355,0.999858,0.003692,0.992475,1.007242,0,0


In [87]:
# #Gera a saida desejada para treinar a rede neural
# df['Long'] = 0
# df['Short'] = 0

# #Percorre o arquivo vendo em quais velas um long teria dado lucro
# for i in range(0, len(df)):
#     stop = df['Low'].iloc[i] - TAM_STOP * df['ATR'].iloc[i]
#     size_stop = df['Close'].iloc[i] - stop + TOTAL_TAXAS
#     take = df['Close'].iloc[i] + size_stop * RISCO_RETORNO + TOTAL_TAXAS
#     for j in range(i+1, len(df)):
#         if df['Low'].iloc[j] <= stop:
#             df.at[df.index[i], 'Long'] = 0
#             break  
#         elif df['High'].iloc[j] > take:
#             df.at[df.index[i], 'Long'] = 1
#             break
    
# #Percorre o arquivo vendo em quais velas um short teria dado lucro
# for i in range(0, len(df)):
#     stop = df['High'].iloc[i] + TAM_STOP * df['ATR'].iloc[i]
#     size_stop = stop - df['Close'].iloc[i] + TOTAL_TAXAS
#     take = df['Close'].iloc[i] - size_stop * RISCO_RETORNO - TOTAL_TAXAS
#     for j in range(i+1, len(df)):
#         if df['High'].iloc[j] >= stop:
#             df.at[df.index[i], 'Short'] = 0
#             break
#         elif df['Low'].iloc[j] < take:
#             df.at[df.index[i], 'Short'] = 1
#             break 

In [88]:
df.dropna(axis=0, how='any', inplace=True)
df.to_csv('./output/Dataframe_2.csv')

In [89]:
#Calcula quando a gente faturaria se entrasse em todas as velas com short e long (Ideia do orientador)

# df['Long'] = 0
# df['Short'] = 0
# df['Equity'] = 0

# sum_long = 0
# for i in range(0, len(df)):
#     #if True: #df['Close'].iloc[i] > df['EMA'+str(EMA_FAST)].iloc[i] and df['EMA'+str(EMA_FAST)].iloc[i] > df['EMA'+str(EMA_SLOW)].iloc[i]:
#     if df['Close'].iloc[i] > df['EMA'+str(EMA_FAST)].iloc[i] and df['EMA'+str(EMA_FAST)].iloc[i] > df['EMA'+str(EMA_SLOW)].iloc[i]:
#         stop = df['Low'].iloc[i] - TAM_STOP * df['ATR'].iloc[i]
#         size_stop = df['Close'].iloc[i] - stop + TOTAL_TAXAS
#         take = df['Close'].iloc[i] + size_stop * RISCO_RETORNO + TOTAL_TAXAS
#         for j in range(i+1, len(df)):
#             if df['Low'].iloc[j] <= stop:
#                 df.at[df.index[i], 'Long'] = -1
#                 break  
#             elif df['High'].iloc[j] > take:
#                 df.at[df.index[i], 'Long'] = RISCO_RETORNO
#                 break
#         sum_long += df['Long'].iloc[i]
#     df.at[df.index[i], 'Equity'] += sum_long

# sum_short = 0
# for i in range(0, len(df)):
#     #if True: #df['Close'].iloc[i] < df['EMA'+str(EMA_FAST)].iloc[i] and df['EMA'+str(EMA_FAST)].iloc[i] < df['EMA'+str(EMA_SLOW)].iloc[i]:
#     if df['Close'].iloc[i] < df['EMA'+str(EMA_FAST)].iloc[i] and df['EMA'+str(EMA_FAST)].iloc[i] < df['EMA'+str(EMA_SLOW)].iloc[i]:
#         stop = df['High'].iloc[i] + TAM_STOP * df['ATR'].iloc[i]
#         size_stop = stop - df['Close'].iloc[i] + TOTAL_TAXAS
#         take = df['Close'].iloc[i] - size_stop * RISCO_RETORNO - TOTAL_TAXAS
#         for j in range(i+1, len(df)):
#             if df['High'].iloc[j] >= stop:
#                 df.at[df.index[i], 'Short'] = -1
#                 break
#             elif df['Low'].iloc[j] < take:
#                 df.at[df.index[i], 'Short'] = RISCO_RETORNO
#                 break 
#         sum_short += df['Short'].iloc[i]
#     df.at[df.index[i], 'Equity'] += sum_short

In [90]:
# fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.2, row_width=[0.2, 0.7])
# fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], showlegend=False, line=dict(width=1.0), opacity=1.0, 
#             increasing_fillcolor='#dadce3', decreasing_fillcolor="#5d606b", increasing_line_color='#434651', decreasing_line_color='#434651'), row=1, col=1)  

# fig.add_trace(go.Scatter(x=df.index, y=df['EMA'+str(EMA_FAST)], marker_color='#FF9800', name='EMA'+str(EMA_FAST), showlegend=False, 
#                     opacity =0.85, line=dict(color="red", width=2, dash='solid')), row=1, col=1)

# fig.add_trace(go.Scatter(x=df.index, y=df['EMA'+str(EMA_SLOW)], marker_color='#FF9800', name='EMA'+str(EMA_SLOW), showlegend=False, 
#                     opacity =0.85, line=dict(color="green", width=2, dash='solid')), row=1, col=1)

# fig.add_trace(go.Scatter(x=df.index, y=df['Equity'], name='Equity', showlegend=False, opacity=1.0, mode='lines', line=dict(color="#787B86", width=1.0, dash='solid')), row=2, col=1)

# fig.update_yaxes(type='linear', row=1, col=1)
# fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    
# fig.update(layout_xaxis_rangeslider_visible=False)
# fig.update_layout(title=TICKET + str(TIMEFRAME) + '   -   ' + INICIO + ' a ' + FINAL, xaxis_tickfont_size=12, xaxis=dict(titlefont_size=14, tickfont_size=12), yaxis=dict(title='Price', 
#             titlefont_size=14, tickfont_size=12), height=850, autosize=True, margin=dict(l=0,r=20, b=20, t=40, pad=0), paper_bgcolor='white', template='simple_white')
# fig.show()